# Cadeias de Markov

*Rodrigo Togneri, 2021-03.*

In [1]:
import numpy as np

## Probabilidades de transição - Versão Dicionário (CONSTRUINDO NA MÃO)

Este é um primeiro exemplo, mais fácil de visualizar do ponto de vista de programação, mas ainda com a desvantagem de não utilizar cálculo matricial.

Primeiro, criemos uma classe para trabalharmos com Cadeias de Markov:

In [2]:
class MarkovChain(object):
    
    def __init__(self, transition_prob):
        # Inicializa uma instância de Cadeia de Markov.
        # Parâmetros:
        # - transition_prob: dict
        #      Um objeto do tipo dicionário representando as probabilidades
        #      de transição na Cadeia de Markov.
        #      Deve estar na seguinte forma:
        #         {'estado1':{'estado1': 0.1, 'estado2': 0.4, 'estado3': 0.5},
        #          'estado2': {...}}
        self.transition_prob = transition_prob
        self.states = list(transition_prob.keys())
    
    def next_state(self, current_state):
        # Retorna o estado de uma variável aleatória na próximo período de 
        # tempo.
        # Parâmetros:
        # - current_state: str
        #      O estado corrente do sistema.
        return np.random.choice(
            self.states, p=[self.transition_prob[current_state][next_state] 
                            for next_state in self.states])
    
    def generate_states(self, current_state, no=10):
        # Retorna os próximos estados do sistema.
        # Parâmetros:
        # - current_state: str
        #      O estado corrente do sistema.
        # - no: int
        #      The number of future states to generate.
        future_states = []
        for i in range(no):
            next_state = self.next_state(current_state)
            future_states.append(next_state)
            current_state = next_state
        return future_states

Agora criemos um dicionário com as probabilidades de transições (exemplo dado em aula, de transição entre canais de vendas):

In [51]:
transition_prob = {'CanalA': {'CanalA': 0.00, 'CanalB': 0.10, 'Venda': 0.20, 'Saida': 0.70},
                   'CanalB': {'CanalA': 0.20, 'CanalB': 0.00, 'Venda': 0.30, 'Saida': 0.50},
                   'Venda':  {'CanalA': 0.00, 'CanalB': 0.00, 'Venda': 1.00, 'Saida': 0.00},
                   'Saida':  {'CanalA': 0.00, 'CanalB': 0.00, 'Venda': 0.00, 'Saida': 1.00}}

Pronto! Agora podemos criar nosso primeiro objeto de Cadeias de Markov:

In [4]:
fluxoDeCanais = MarkovChain(transition_prob=transition_prob)

Com o objeto criado, podemos invocar uma geração aleatória de próximo estado dado um estado corrente:

In [5]:
fluxoDeCanais.next_state(current_state='CanalA')

'Saida'

In [6]:
fluxoDeCanais.next_state(current_state='CanalB')

'CanalA'

Note que quando o estado corrente é o de Venda (ou o de Saída , que são estados terminais), o próximo estado é sempre o próprio estado corrente.

In [7]:
fluxoDeCanais.next_state(current_state='Venda')

'Venda'

Podemos também gerar simulações sequenciais de uma cadeia de markov, partindo de um estado corrente:

In [20]:
fluxoDeCanais.generate_states(current_state='CanalA', no=10)

['Venda',
 'Venda',
 'Venda',
 'Venda',
 'Venda',
 'Venda',
 'Venda',
 'Venda',
 'Venda',
 'Venda']

## Probabilidades de transição - Versão Array (melhor) (CONSTRUINDO DO ZERO)

Agora vamos novamente construir um exemplo do zero, só que agora fazendo uso de matriz de transição (array) ao invés de um dicionário.

Primeiro, criemos uma classe que lide com Cadeias de Markov:

In [39]:
class MarkovChain(object):
    def __init__(self, transition_matrix, states):
        # Inicializa uma instância de Cadeia de Markov.
        # Parâmetros:
        # - transition_prob: 2-D array
        #     Um array 2-D repressentando a matriz de transição.
        # - states: 1-D array
        #     Um array representando os estados possíveis da Cadeia de Markov
        #     (precisa ser na mesma ordem de estados na matriz de transição).
        self.transition_matrix = np.atleast_2d(transition_matrix)
        self.states = states
        self.index_dict = {self.states[index]: index for index in 
                           range(len(self.states))}
        self.state_dict = {index: self.states[index] for index in
                           range(len(self.states))}
 
    def next_state(self, current_state):
        # Retorna o estado de uma variável aleatória na próximo período de 
        # tempo.
        # Parâmetros:
        # - current_state: str
        #      O estado corrente do sistema.
        return np.random.choice(
         self.states, 
         p=self.transition_matrix[self.index_dict[current_state], :]
        )
 
    def generate_states(self, current_state, no=10):
        # Retorna os próximos estados do sistema.
        # Parâmetros:
        # - current_state: str
        #      O estado corrente do sistema.
        # - no: int
        #      The number of future states to generate.
        future_states = []
        for i in range(no):
            next_state = self.next_state(current_state)
            future_states.append(next_state)
            current_state = next_state
        return future_states
    
    def systemEvolution(self, initial_globalState, steps):
        tmpCurrMatrix = self.transition_matrix
        globalStates = []
        globalStates.append(initial_globalState)
        tmpNewGlobalState = initial_globalState
        for i in range(steps-1):
            tmpNewGlobalState = np.matmul(tmpNewGlobalState, tmpCurrMatrix).tolist()
            globalStates.append(tmpNewGlobalState)
            tmpCurrMatrix = np.matmul(tmpCurrMatrix, fluxoDeCanais.transition_matrix)
        return globalStates

Criemos uma matriz de transição (usado o mesmo exemplo dado em aula, de fluxo de transiÇão de canais de vendas):

In [53]:
transition_matrix = [[0.00, 0.10, 0.20, 0.70],
                     [0.20, 0.00, 0.30, 0.50],
                     [0.00, 0.00, 1.00, 0.00],
                     [0.00, 0.00, 0.00, 1.00]]

Pronto! Agora podemos criar um objeto da classe criada:

In [41]:
fluxoDeCanais = MarkovChain(transition_matrix=transition_matrix,
                            states=['CanalA', 'CanalB', 'Venda', 'Saida'])

Veja o que index_dict e state_dict armazenam:

In [42]:
fluxoDeCanais.index_dict

{'CanalA': 0, 'CanalB': 1, 'Venda': 2, 'Saida': 3}

In [43]:
fluxoDeCanais.state_dict

{0: 'CanalA', 1: 'CanalB', 2: 'Venda', 3: 'Saida'}

Da mesma forma, podemos simular aleatoriamente o próximo estado à partir do estado corrente:

In [44]:
fluxoDeCanais.next_state(current_state='CanalA')

'Saida'

In [45]:
fluxoDeCanais.next_state(current_state='CanalB')

'Venda'

In [46]:
fluxoDeCanais.next_state(current_state='Venda')

'Venda'

E gerar uma simulação de sequência de estados a partir de um estado corrente:

In [47]:
fluxoDeCanais.generate_states(current_state='CanalA', no=10)

['Saida',
 'Saida',
 'Saida',
 'Saida',
 'Saida',
 'Saida',
 'Saida',
 'Saida',
 'Saida',
 'Saida']

Vamos agora criar uma estimativa de evolução do sistema como um todo (aqui começamos a enxergar a imporância de se utilizar cálculo matricial, justamente porque a evolução ocorre pela multiplicação iterativa da matriz de transição):

In [55]:
steps = 10
initial_globalState = [1, 0, 0, 0]

In [56]:
tmpCurrMatrix = fluxoDeCanais.transition_matrix
globalStates = []
globalStates.append(initial_globalState)
tmpNewGlobalState = initial_globalState
for i in range(steps-1):
    tmpNewGlobalState = np.matmul(tmpNewGlobalState, tmpCurrMatrix).tolist()
    globalStates.append(tmpNewGlobalState)
    tmpCurrMatrix = np.matmul(tmpCurrMatrix, fluxoDeCanais.transition_matrix)

In [57]:
globalStates

[[1, 0, 0, 0],
 [0.0, 0.1, 0.2, 0.7],
 [0.0, 0.0020000000000000005, 0.234, 0.764],
 [8.000000000000003e-06, 0.0, 0.234692, 0.7653],
 [3.200000000000002e-09, 0.0, 0.2346938768, 0.76530612],
 [0.0, 1.2800000000000014e-13, 0.234693877550976, 0.765306122448896],
 [0.0, 1.0240000000000014e-18, 0.23469387755102042, 0.7653061224489796],
 [1.638400000000003e-24, 0.0, 0.23469387755102042, 0.7653061224489796],
 [2.6214400000000064e-31, 0.0, 0.23469387755102042, 0.7653061224489796],
 [0.0, 4.1943040000000123e-39, 0.23469387755102042, 0.7653061224489796]]

Mas como já temos o código acima implementado como método na classe MarkovChain, podemos apenas invocar o método para obtermos o mesmo resultado:

In [58]:
fluxoDeCanais.systemEvolution(initial_globalState, steps)

[[1, 0, 0, 0],
 [0.0, 0.1, 0.2, 0.7],
 [0.0, 0.0020000000000000005, 0.234, 0.764],
 [8.000000000000003e-06, 0.0, 0.234692, 0.7653],
 [3.200000000000002e-09, 0.0, 0.2346938768, 0.76530612],
 [0.0, 1.2800000000000014e-13, 0.234693877550976, 0.765306122448896],
 [0.0, 1.0240000000000014e-18, 0.23469387755102042, 0.7653061224489796],
 [1.638400000000003e-24, 0.0, 0.23469387755102042, 0.7653061224489796],
 [2.6214400000000064e-31, 0.0, 0.23469387755102042, 0.7653061224489796],
 [0.0, 4.1943040000000123e-39, 0.23469387755102042, 0.7653061224489796]]

Veja o que acontece quando calculamos os autovalores e autovetores da transposta da matriz de transição (os estados terminais sempre apresentam autovalor 1 e autovetores com um valor 1 e o restante de zeros):

In [95]:
np.linalg.eig(np.transpose(fluxoDeCanais.transition_matrix))

(array([ 1.        ,  1.        ,  0.14142136, -0.14142136]),
 array([[ 0.        ,  0.        ,  0.55588453, -0.79249389],
        [ 0.        ,  0.        ,  0.39306972,  0.5603778 ],
        [ 1.        ,  0.        , -0.26683383, -0.00842333],
        [ 0.        ,  1.        , -0.68212043,  0.24053941]]))

## Pagerank

Realizando o exemplo da lousa:

In [59]:
transition_matrix = [[0, 1/2, 1/2, 0],
                     [0, 0, 1/2, 1/2],
                     [0, 0, 0, 1],
                     [1/3, 1/3, 1/3, 1]]

In [60]:
transition_matrix_T = np.transpose(transition_matrix)

In [62]:
np.linalg.eig(transition_matrix_T)

(array([ 1.52869933+0.j        , -0.1506424 +0.46584615j,
        -0.1506424 -0.46584615j, -0.22741452+0.j        ]),
 array([[ 0.19282539+0.j        , -0.12001557-0.3711358j ,
         -0.12001557+0.3711358j , -0.44872832+0.j        ],
        [ 0.25589384+0.j        , -0.44293802-0.13789657j,
         -0.44293802+0.13789657j,  0.53785846+0.j        ],
        [ 0.33959043+0.j        , -0.43775111+0.33583846j,
         -0.43775111-0.33583846j, -0.64469237+0.j        ],
        [ 0.88431615+0.j        ,  0.57291485+0.j        ,
          0.57291485-0.j        ,  0.306142  +0.j        ]]))

In [71]:
l = np.linalg.eig(transition_matrix_T)[1][:,0].tolist()

In [74]:
l

[(0.1928253929445729+0j),
 (0.25589384245112284+0j),
 (0.3395904325900821+0j),
 (0.8843161467058327+0j)]